In [ ]:
# importing numpy and pandas
import numpy as np
import pandas as pd
import sklearn as sk

In [ ]:
# reading both csv's data and saving in a variables named movie and credit
movies = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
# printing top 5 rows of movie csv
movies.head()

In [ ]:
# printing top 5 rows of credit 
credit.head()

In [ ]:
# merging both db based on title
movies = movies.merge(credit, on='title')

In [ ]:
type(movies)


In [ ]:
print(movies.columns)


In [ ]:
movies = movies[['genres', 'title', 'overview', 'keywords', 'crew', 'cast', 'movie_id']]

In [ ]:
#check missing data

movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
# check duplicate data
movies.duplicated().sum()

In [ ]:
print(type(movies['genres'].iloc[0]))
print(movies['genres'].iloc[0])


In [ ]:
# data cleaning of genres
movies.iloc[0].genres

In [ ]:
# this is helper function to clean genre column and get only genre name 
# ast.literal_eval is used for changing string indices in integer
import ast

def convert(obj):
    if isinstance(obj, str):
        try:
            data = ast.literal_eval(obj)
            return [d['name'] for d in data]  # extract "name" field
        except:
            return []
    elif isinstance(obj, list):
        return [d['name'] if isinstance(d, dict) else d for d in obj]
    else:
        return []
  

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
# helper function fot top 3 cast

def convert_cast(obj):
    if isinstance(obj, str):
        try:
            data = ast.literal_eval(obj)
            return [d['name'] for d in data[:3]] 
        except:
            return []
    elif isinstance(obj, list):
        return [d['name'] if isinstance(d, dict) else d for d in obj[:3]]
    else:
        return []
  


In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [ ]:
# helper function to extractt director name from crew

def director(obj):
    try:
        data = ast.literal_eval(obj) if isinstance(obj, str) else obj
        for d in data:
            if d.get('job') == 'Director':
                return d['name']
        return None
    except Exception as e:
        return None




In [ ]:
movies['crew'] = movies['crew'].apply(director)

In [ ]:
# change overview colunm from string to list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
#removing spaces
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
# movies['crew'] = movies['crew'].apply(lambda x: ["".join(x).replace(" ", "")])
movies['crew'] = movies['crew'].apply(
    lambda x: x.replace(" ", "") if isinstance(x, str) else ""
)



In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew'] 

In [ ]:
new_df = movies[['title', 'tags', 'movie_id']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'][2]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

In [187]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [193]:
recommend('Iron Man')

Iron Man 3
Iron Man 2
Avengers: Age of Ultron
The Helix... Loaded
Teenage Mutant Ninja Turtles II: The Secret of the Ooze


In [184]:
new_df.iloc[942].title

'The Book of Life'

In [195]:
import pickle

In [196]:
pickle.dump(new_df, open('movies.pkl','wb'))

In [197]:
pickle.dump(similarity,open('similarity.pkl','wb'))